In [1]:
import numpy as np
from scipy.stats import binom

In [7]:
# Parameters 

h = 1/7
pho = 2/3
gamma = 2/3
R = 19
T = 7*R

In [8]:
#  Effective Sampling Size 

def ESS(W):                 # Takes an array as argument
    return 1/np.sum(W**2)

In [14]:
# Mutation transition

def mutation(theta, k, S, E, I, R):
    #
    M = theta[k:k+4]
    beta = 1/n * np.dot(M, I)
    B = np.random.binomial(S, 1 - np.exp(-h*beta)) 
    C = np.random.binomial(E, 1 - np.exp(-h*pho)) 
    D = np.random.binomial(I, 1 - np.exp(-h*gamma)) 
    S_new = S - B
    E_new = E + B - C
    I_new = I + C - D
    R_new = R - D
    return S_new, E_new, I_new, R_new

In [15]:
# Likelihood of y knowing theta

def likelihood(y, theta, k, S, E, I, R):
    C = np.zeros(shape=(7, 4))
    S_new, E_new, I_new, R_new =  S, E, I, R
    for i in range(7):
        S_new[k], E_new[k], I_new[k], R_new[k] = mutation(theta, k, S_new, E_new, I_new, R_new)
        C[i] = np.random.binomial(E_new, 1 - np.exp(-h*pho))
    return(binom.pmf(y, np.sum(C, axis=0), theta[16:]))

In [ ]:
# Initial settings
S_init = np.array([948, 1689, 3466, 1894])
E_init = np.array([0, 0, 0, 0])
I_init = np.array([1, 1, 1, 1])
R_init = np.array([0, 0, 0, 0])

def BootstrapFilter(y, N, R=19, ESS_min):
    S, E, I, R = S_init, E_init, I_init, R_init
    thetas = []
    w_init = np.zeros((N, 4))
    for n in range(N):
        theta = np.concatenate(np.random.gamma(5, 1, 16), np.random.normal(0.5, 0.5, 4), axis=None)
        thetas.append(theta)
        w_init[n] = likelihood(y, theta, S, E, I, R)
    W_init = np.zeros((N, 4))
    for n in range(N):
        W_init[n] = np.divide(w_init[n], np.sum(w_init, axis=0))
    W = W_init
    X = 
    for r in range(R):
        for k in range(4):
            if ESS(W[:, k]) < ESS_min:
                A = np.random.choice(np.arange(1, N), p=W[:, k])
                w_hat = np.ones(N)
            else:
                A = np.arange(1, N)
                w_hat = w
        for n in range(N):           
            